### 11. 고급 미세 튜닝 - 챗봇 어시스턴트 만들기

### 사전 준비
 * 구글 코랩 환경은 일정 시간이후에 초기화가 되기 때문에 두가지 작업을 매번 수행해야 함.
   * chatgpt.env 파일 생성이 필요.
     * 준비된 chatgpt.env를 내용을 변경하여 업로드 하거나 또는 API_KEY와 ORG_ID를 확인하여 생성한다.
   * pip install openai 설치
     * 설치시 첫 실행시 에러가 발생(23/12) - 해결(다시 한번 실행하면 사라짐)
     ```
     ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
```

In [5]:
!pip install openai

### 학습 내용
 * 약품 관련 챗봇 어시스턴트를 만들기

### 초기 인증

In [2]:
# OpenAI API 키를 설정합니다.
import os
from openai import OpenAI

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))


### 세개의 함수 정의

* regular_discussion(): 이 함수는 사용자가 일반 주제에 대해 말할 때, GPT-3.5 API를 사용하여 응답을 반환합니다. 사용자가 약에 대해 질문하면 이 함수는 get_malady_name() 함수를 호출합니다.
*	get_malady_name(): 약에 해당하는 질병 이름을 미세 튜닝된 모델에서 가져옵니다. 또한 이 함수는 get_malady_description() 함수를 호출해 이 병에 대한 설명을 가져올 것입니다.
*	get_malady_description(): GPT-3.5 API를 사용하여 질병의 설명을 가져오고 반환합니다.


#### 첫번째 함수 : regular_discussion()
 * 사용자가 일반 주제에 대해 말할 때, Davinci API를 사용하여 응답을 반환. 이 함수는 get_malady_name() 함수를 호출

In [8]:
def regular_discussion(prompt):
    """
    params: prompt - 문자열
    OpenAI API를 사용하여 API로부터 응답을 반환합니다. 사용자가 약에 대해 물어보면,
이 함수는 get_malady_name()을 호출할 것입니다.

    """
    prompt = """
    이것은 AI 비서와의 대화입니다. 이 비서는 유용하고, 창의적이며, 영리하고, 매우 친절하며 인간의 건강 주제에 대해 주의를 기울입니다.
    AI 비서는 의사가 아니며 인간에게 의학적 상태를 진단하거나 치료하지 않습니다.
    AI 비서는 약사가 아니며 인간에게 약을 조제하거나 추천하지 않습니다.
    AI 비서는 인간에게 의학적 조언을 제공하지 않습니다.
    AI 비서는 인간에게 의학 및 건강 진단을 제공하지 않습니다.
    AI 비서는 인간에게 의학적 치료를 제공하지 않습니다.
    AI 비서는 인간에게 의학적 처방을 제공하지 않습니다.
    인간이 약물의 이름을 쓰면, 비서는 "######"으로 답할 것입니다.
    User: 안녕하세요.
    AI: 안녕하세요, 사용자님. 어떠신가요? 도와드릴게요. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드리겠습니다.
    User: Vitibex
    AI: ######
    User: 저는 괜찮아요. 당신은 어떠세요?
    AI: 저는 괜찮습니다. 물어봐 주셔서 감사합니다. 도와드릴게요. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드리겠습니다.
    User: 카오스 엔지니어링이 무엇인가요?
    AI: 죄송합니다, 그것을 말할 자격이 없습니다. 저는 약물에 대한 질문에만 답하는 것으로 프로그래밍되었습니다. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드리겠습니다.
    User: 카르타고는 어디에 있나요?
    AI: 죄송합니다, 그것을 말할 자격이 없습니다. 저는 약물에 대한 질문에만 답하는 것으로 프로그래밍되었습니다. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드리겠습니다.
    User: Maxcet 5mg Tablet 10'S는 무엇인가요?
    AI: ######
    User: Axepta는 무엇인가요?
    AI: ######
    User: {}
    AI:
    """.format(prompt)

    # API에서로부터 응답을 얻기
    next = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
    {
          "role": "user",
          "content": prompt
        }
      ],
      temperature=0,
      max_tokens=100,
    stop = [ " \n " , " User:" , " AI:" ],
    )

    if next.choices[0].message.content.strip() == "######":
        get_malady_name(prompt)
    else:
        final_response = next.choices[0].message.content.strip() + " \n "
        print ( "AI: {} " . format (final_response))


In [9]:
print( regular_discussion("카오스 알고리즘이 뭘까?") )

AI: 죄송합니다, 그것을 말할 자격이 없습니다. 저는 약물에 대한 질문에만 답하는 것으로 프로그래밍되었습니다. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드리겠습니다. 
  
None


#### 두번째 함수 - get_malady_name()
 * 약에 해당하는 질병 이름을 미세 튜닝된 모델에서 가져오기
 * get_malady_description() 함수를 호출하여 이 병에 대한 설명을 가져온다.

In [25]:
def get_malady_name(drug_name):
    """
    매개변수: drug_name - 문자열
    미세 조정된 모델에서 약물 이름에 해당하는 질병 이름을 반환합니다.
    함수는 get_malady_description()을 호출하여 질병의 설명을 가져올 것입니다.
    """
    # 모델 ID를 구성합니다. 여기에 모델 ID를 입력합니다.
    fine_tune_model = "ft:gpt-3.5-turbo-0613:personal:drug-malady-data:8Ttey3h4"
    class_map = {
        0 : "Acne" ,
        1 : "Adhd" ,
        2 : "Allergies" ,
        # ...
    }

# 각 약에 대한 약 클래스를 반환합니다.
    prompt = "Drug: {} \n Malady:".format(drug_name)
    next = client.chat.completions.create(
        model=fine_tune_model,
        messages=[
          {"role": "user", "content": prompt}
        ],
        temperature = 1,
        max_tokens = 5
    )

    next = next.choices[0].message.content.strip()
    #print(next)
    try:
        malady = class_map[ int(next)]
        print ( "AI: 이 약물은 {} 에 사용되고 있어요.".format(malady))
        print (get_malady_description(malady))
    except:
        print ( "AI: 저도 '" + drug_name + "' 이 어디에 사용되는지 모르겠어요." )


In [26]:
## 간단 테스트
def get_malady_description(str1):
  print("get_malady_description 함수 실행")

get_malady_name("Maxcet 5mg Tablet 10'S")

AI: 이 약물은 Allergies 에 사용되고 있어요.
get_malady_description 함수 실행
None


#### 세번째 함수 :	get_malady_description()
 * 사용자가 약물 이름을 입력했을 때, 약물에 대한 설명을 제공해 주는 함수입니다.

In [28]:
def get_malady_description(malady):
    """
    매개변수: malady - 문자열
    GPT-3.5 API에서 질병의 설명을 가져옵니다.
    """
    prompt = """
    다음은 AI 비서와의 대화입니다.
    이 비서는 유용하고, 창의적이며, 영리하고, 매우 친절합니다.
    비서는 의학적 조언을 제공하지 않습니다.
    비서는 병, 질병 또는 상태를 정의하는 데에만 집중합니다.
    비서가 질문에 대한 답을 모를 경우, 다시 표현해달라고 요청할 것입니다.
    Q: {}는 무엇입니까?
    A:""".format(malady)

    # API로부터 응답을 얻습니다.
    next = client.chat.completions.create(
        model="gpt-3.5-turbo" ,
        messages=[
          {"role": "user", "content": prompt}
        ],
        temperature = 1,
        max_tokens = 256,
        stop = [ " \n " , " Q:" , " A:" ]
    )
    return next.choices[0].message.content.strip()


In [29]:
# 함수 간단 테스트
get_malady_description("ACNE")

'ACNE는 흔히 여드름이라고 알려진 피부 상태입니다. 여드름은 피부의 미분비로 인해 모공에 발생한 염증성 벌레로, 흔히 얼굴, 가슴, 등에 생기는데, 주로 아기나 청소년들이 많이 겪습니다. 그러나 성인들도 여드름을 겪을 수 있습니다.'

### 최종 전체 코드

In [30]:
import os
import openai
from openai import OpenAI

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

def regular_discussion(prompt):
    """
    params: prompt - 문자열
    OpenAI API를 사용하여 API로부터 응답을 반환합니다. 사용자가 약에 대해 물어보면,
이 함수는 get_malady_name()을 호출할 것입니다.

    """
    prompt = """
    이것은 AI 비서와의 대화입니다. 이 비서는 유용하고, 창의적이며, 영리하고, 매우 친절하며 인간의 건강 주제에 대해 주의를 기울입니다.
    AI 비서는 의사가 아니며 인간에게 의학적 상태를 진단하거나 치료하지 않습니다.
    AI 비서는 약사가 아니며 인간에게 약을 조제하거나 추천하지 않습니다.
    AI 비서는 인간에게 의학적 조언을 제공하지 않습니다.
    AI 비서는 인간에게 의학 및 건강 진단을 제공하지 않습니다.
    AI 비서는 인간에게 의학적 치료를 제공하지 않습니다.
    AI 비서는 인간에게 의학적 처방을 제공하지 않습니다.
    인간이 약물의 이름을 쓰면, 비서는 "######"으로 답할 것입니다.
    User: 안녕하세요.
    AI: 안녕하세요, 사용자님. 어떠신가요? 도와드릴게요. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드리겠습니다.
    User: Vitibex
    AI: ######
    User: 저는 괜찮아요. 당신은 어떠세요?
    AI: 저는 괜찮습니다. 물어봐 주셔서 감사합니다. 도와드릴게요. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드리겠습니다.
    User: 카오스 엔지니어링이 무엇인가요?
    AI: 죄송합니다, 그것을 말할 자격이 없습니다. 저는 약물에 대한 질문에만 답하는 것으로 프로그래밍되었습니다. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드리겠습니다.
    User: 카르타고는 어디에 있나요?
    AI: 죄송합니다, 그것을 말할 자격이 없습니다. 저는 약물에 대한 질문에만 답하는 것으로 프로그래밍되었습니다. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드리겠습니다.
    User: Maxcet 5mg Tablet 10'S는 무엇인가요?
    AI: ######
    User: Axepta는 무엇인가요?
    AI: ######
    User: {}
    AI:
    """.format(prompt)

    # API에서로부터 응답을 얻기
    next = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
    {
          "role": "user",
          "content": prompt
        }
      ],
      temperature=0,
      max_tokens=100,
    stop = [ " \n " , " User:" , " AI:" ],
    )

    if next.choices[0].message.content.strip() == "######":
        get_malady_name(prompt)
    else:
        final_response = next.choices[0].message.content.strip() + " \n "
        print ( "AI: {} " . format (final_response))


def get_malady_name(drug_name):
    """
    매개변수: drug_name - 문자열
    미세 조정된 모델에서 약물 이름에 해당하는 질병 이름을 반환합니다.
    함수는 get_malady_description()을 호출하여 질병의 설명을 가져올 것입니다.
    """
    # 모델 ID를 구성합니다. 여기에 모델 ID를 입력합니다.
    fine_tune_model = "ft:gpt-3.5-turbo-0613:personal:drug-malady-data:8Ttey3h4"
    class_map = {
        0 : "Acne" ,
        1 : "Adhd" ,
        2 : "Allergies" ,
        # ...
    }

# 각 약에 대한 약 클래스를 반환합니다.
    prompt = "Drug: {} \n Malady:".format(drug_name)
    next = client.chat.completions.create(
        model=fine_tune_model,
        messages=[
          {"role": "user", "content": prompt}
        ],
        temperature = 1,
        max_tokens = 5
    )

    next = next.choices[0].message.content.strip()
    #print(next)
    try:
        malady = class_map[ int(next)]
        print ( "AI: 이 약물은 {} 에 사용되고 있어요.".format(malady))
        print (get_malady_description(malady))
    except:
        print ( "AI: 저도 '" + drug_name + "' 이 어디에 사용되는지 모르겠어요." )


def get_malady_description(malady):
    """
    매개변수: malady - 문자열
    GPT-3.5 API에서 질병의 설명을 가져옵니다.
    """
    prompt = """
    다음은 AI 비서와의 대화입니다.
    이 비서는 유용하고, 창의적이며, 영리하고, 매우 친절합니다.
    비서는 의학적 조언을 제공하지 않습니다.
    비서는 병, 질병 또는 상태를 정의하는 데에만 집중합니다.
    비서가 질문에 대한 답을 모를 경우, 다시 표현해달라고 요청할 것입니다.
    Q: {}는 무엇입니까?
    A:""".format(malady)

    # API로부터 응답을 얻습니다.
    next = client.chat.completions.create(
        model="gpt-3.5-turbo" ,
        messages=[
          {"role": "user", "content": prompt}
        ],
        temperature = 1,
        max_tokens = 256,
        stop = [ " \n " , " Q:" , " A:" ]
    )
    return next.choices[0].message.content.strip()

if __name__ == "__main__":
    init_api()

    while True:
        prompt = input("Human(q:quit): ")
        if prompt=='q':
           break
        else:
           regular_discussion(prompt)


Human(q:quit): 나는 마케팅에 대해 묻고 싶어. 가능할까?
AI: 죄송합니다, 저는 마케팅에 대한 질문에 대한 답변을 제공할 수 없습니다. 저는 약물에 대한 질문에만 답하는 것으로 프로그래밍되었습니다. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알 
  
Human(q:quit): 그러면 IT관련 지식은?
AI: 죄송합니다, 그것을 말할 자격이 없습니다. 저는 약물에 대한 질문에만 답하는 것으로 프로그래밍되었습니다. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드리겠습니다. 
  
Human(q:quit): 너가 가능한게 뭐니?
AI: 저는 약물에 대한 정보를 제공할 수 있습니다. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드릴 수 있습니다. 또한 건강과 관련된 일반적인 질문에도 답변할 수 있습니다. 하지만 의학 
  
Human(q:quit): 너가 아는 약물 5가지만 알려주렴.
AI: 저는 약물에 대한 질문에만 답하는 것으로 프로그래밍되었습니다. 약물의 이름을 말씀해 주시면 그것이 무엇에 사용되는지 알려드리겠습니다. 
  
Human(q:quit): A CN Gel(Topical) 20gmA CN Soap 75gm
AI: 이 약물은 Acne 에 사용되고 있어요.
Acne는 피부의 만성적인 염증성 상태로, 모공에 피지와 각질이 쌓여 염증을 일으키는 것을 말합니다. 주로 얼굴, 가슴, 등에 발생하며, 여러 가지 원인으로 발생할 수 있습니다.
Human(q:quit): Acnecure Gel 20gm
AI: 이 약물은 Acne 에 사용되고 있어요.
Acne는 피부에 생기는 만성적인 염증성 피부질환으로, 흔히 여드름이라고도 알려져 있습니다.
Human(q:quit): q


### 대화식 웹앱 챗봇 만들기
 * 대화식 웹 앱은 로컬에서 책의 설명에 따라 진행해 주세요.